In [88]:
import pandas as pd
import numpy as np
import pprint

from bs4 import BeautifulSoup

import json

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 100)

In [2]:
#df = pd.read_json('/Users/aeevr/Documents/CodeForSanJose/heartofthevalley/artwork-data/consolidated-artworks.json', orient='columns')

In [3]:
fname='/Users/aeevr/Documents/CodeForSanJose/heartofthevalley/artwork-data/consolidated-artworks.json'

# Flatten json to fit in pandas dataframe

In [4]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df

#load json object
with open(fname) as f:
    d = json.load(f)

#lets put the data into a pandas df
#clicking on consolidated-artworks.json under "Input Files"
#tells us parent node WE WANT TO EDIT is 'features'
artwrk = json_normalize(d['features'])
artwrk.head(len(artwrk))

,addToMap,errors,geometry.coordinates,geometry.type,id,properties.address,properties.artist,properties.city,properties.country,properties.crossStreet,properties.description,properties.image,properties.postalCode,properties.sourceOfInformation,properties.sourceURL,properties.state,properties.title,properties.website,type
0,NaN,NaN,"[-121.88887, 37.32718]",Point,10667523,"488 Almaden Blvd, Ste 1800",<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN,Feature
1,NaN,NaN,"[-121.97798, 37.30554]",Point,23925665,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN,Feature
2,NaN,NaN,"[-121.88582, 37.33022]",Point,31154388,396 S 1st St.,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/,Feature
3,NaN,NaN,"[-121.89577, 37.35107]",Point,38393353,200 E Taylor St,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN,Feature
4,NaN,NaN,"[-121.84637, 37.34036]",Point,45973257,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN,Feature
5,NaN,NaN,"[-121.9139669, 37.3522017]",Point,58144474,1701 Airport Boulevard,Angela Buenning Filo,San Jose,NaN,NaN,Bangalore and Silicon Valley are distant yet interconnected. This photographic installation exp...,http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000...,95110,http://sanjoseca.gov/Facilities,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-CONNECTED-Silicon-Valley-Bang-409,CA,CONNECTED: Silicon Valley + Bangalore,NaN,Feature
6,NaN,NaN,"[-121.885655, 37.333667]",Point,63319839,101 Paseo De San Antonio,Jun Kaneko,San Jose,NaN,NaN,"The artist, Jun Kaneko, calls these curvilinear ceramic pieces “dangos,” meaning dumplings in Ja...",http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000...,95113,http://sanjoseca.gov/Facilities,http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Convergence-San-Jose-Repertor-418,CA,Convergence,NaN,Feature
7,NaN,NaN,"[-121.88986, 37.33567]",Point,72225815,30 S 1st St.,<a href='http://www.andradehemming.com/' target='_blank'>Jeffrey Andrade Hemming</a>,San José,United States,at Fountain Alley and S 1st St.,This mural was produced by <a href='http://www.phantomgalleries.com/' target='_blank'>Phantom Ga...,NaN,95113,NaN,NaN,CA,Phylum of the Free,http://www.andradehemming.com/,Feature
8,NaN,NaN,"[-121.90088, 37.30778]",Point,76443136,1133 Lincoln Ave,<a href='http://www.gemellosmurals.com/' target='_blank'>Lila Gemellos</a>,San José,United States,NaN,"Life in Willow Glen pays homage to the past and present heritage, memories and business culture....",NaN,95125,NaN,NaN,CA,Life in Willow Glen,NaN,Feature
9,NaN,NaN,"[-121.81454, 37.32341]",Point,76520329,2200 Eastridge Loop,<a href='http://www.aarondelacruz.com/' target='_blank'>Aaron De La Cruz</a>,San José,United States,at Eastridge Loop,This mural is part of a collection of murals at Eastridge Mall. <a href='http://eastridgecenter...,NaN,95122,NaN,NaN,CA,Eastridge Mall Mural - Aaron De La Cruz,http://www.aarondelacruz.com/,Feature


# edit 'properties.artist' column keeping text tag as name and putting attrs into 'properties.artist_site'

In [6]:
soup = BeautifulSoup(artwrk['properties.artist'].iloc[0],'lxml')

In [9]:
tmp_col = artwrk['properties.artist']
artist_name = []
artist_site = []

for i in range(len(tmp_col)):
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')
    #print(soup)
    tags = soup.find_all('a')
    #print(tags)
    #print(len(tags))
    if (len(tags)==1): 
        tags = soup.find('a')
        a_n = tags.text
        a_s = tags.attrs['href']
    elif (len(tags)>1):
        for t in tags:
            a_n = t.text
            a_s = t.attrs['href']
            #print(t.text, t.attrs['href'])
    elif (len(tags)==0):
        a_n = tmp_col[i]
        a_s = ''
    #print('a_n:',a_n,' a_s:',a_s,'\n\n')
    artist_name.append(a_n)
    artist_site.append(a_s)
        
artist_df = pd.concat([artwrk['properties.artist'],
                       pd.DataFrame(artist_name,columns=['artist_name']),
                       pd.DataFrame(artist_site,columns=['artist_site'])],axis=1)

In [10]:
artist_df.head(len(artist_df))

,properties.artist,artist_name,artist_site
0,<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,J.Duh,https://www.instagram.com/j.duh_/
1,Carlos Rodriguez,Carlos Rodriguez,
2,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,Mike Borja,http://mikeborja.tumblr.com/
3,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,Sam Rodriguez,http://samrodriguezart.com/
4,Gloria Bornstein,Gloria Bornstein,
5,Angela Buenning Filo,Angela Buenning Filo,
6,Jun Kaneko,Jun Kaneko,
7,<a href='http://www.andradehemming.com/' target='_blank'>Jeffrey Andrade Hemming</a>,Jeffrey Andrade Hemming,http://www.andradehemming.com/
8,<a href='http://www.gemellosmurals.com/' target='_blank'>Lila Gemellos</a>,Lila Gemellos,http://www.gemellosmurals.com/
9,<a href='http://www.aarondelacruz.com/' target='_blank'>Aaron De La Cruz</a>,Aaron De La Cruz,http://www.aarondelacruz.com/


In [20]:
artwrk_cpy = artwrk.copy()
artwrk_cpy['properties.artist'] = artist_name
artwrk_cpy['properties.artist_site'] = artist_site
print(artwrk_cpy.to_json(orient='index'))

{"0":{"addToMap":null,"errors":null,"geometry.coordinates":[-121.88887,37.32718],"geometry.type":"Point","id":10667523,"properties.address":"488 Almaden Blvd, Ste 1800","properties.artist":"J.Duh","properties.city":"San Jos\u00e9","properties.country":"United States","properties.crossStreet":null,"properties.description":"This mural was painted by <a href='https:\/\/www.instagram.com\/j.duh_\/' target='_blank'>J.Duh<\/a>, and produced in collaboration with <a href='https:\/\/www.exhibitiondistrict.com\/' target='_blank'>Exhibition District<\/a>. It is located on the exterior of the PWC Building on Balbach St. <br\/> Source: Exhibition District, https:\/\/www.exhibitiondistrict.com\/","properties.image":null,"properties.postalCode":"95113","properties.sourceOfInformation":null,"properties.sourceURL":null,"properties.state":"CA","properties.title":"San Jos\u00e9","properties.website":null,"type":"Feature","properties.artist_site":"https:\/\/www.instagram.com\/j.duh_\/"},"1":{"addToMap":n

In [14]:
artwrk_cpy.head()

,addToMap,errors,geometry.coordinates,geometry.type,id,properties.address,properties.artist,properties.city,properties.country,properties.crossStreet,properties.description,properties.image,properties.postalCode,properties.sourceOfInformation,properties.sourceURL,properties.state,properties.title,properties.website,type,properties.artist_site
0,NaN,NaN,"[-121.88887, 37.32718]",Point,10667523,"488 Almaden Blvd, Ste 1800",J.Duh,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN,Feature,https://www.instagram.com/j.duh_/
1,NaN,NaN,"[-121.97798, 37.30554]",Point,23925665,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN,Feature,
2,NaN,NaN,"[-121.88582, 37.33022]",Point,31154388,396 S 1st St.,Mike Borja,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/,Feature,http://mikeborja.tumblr.com/
3,NaN,NaN,"[-121.89577, 37.35107]",Point,38393353,200 E Taylor St,Sam Rodriguez,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN,Feature,http://samrodriguezart.com/
4,NaN,NaN,"[-121.84637, 37.34036]",Point,45973257,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN,Feature,


# title column looks clean

In [220]:
artwrk['properties.title']

0                                                                                                 San José
1                                                                Nothing Is Too Good for the Working Class
2                                                                                           Guardian Tiger
3                                                                                       Kokeshis de San Jo
4                                                                              A Monument to the Last Barn
5                                                                    CONNECTED: Silicon Valley + Bangalore
6                                                                                              Convergence
7                                                                                       Phylum of the Free
8                                                                                      Life in Willow Glen
9                                    

In [11]:
artwrk['properties.title'].iloc[41]

'Nunca eres lo suficientemente fuerte para no necesitar ayuda (You are never strong enough where you don’t need help) -- César Chávez'

In [119]:
soup = BeautifulSoup(nycphil['properties.description'].iloc[0],'lxml')

In [120]:
soup.text


'This mural was painted by J.Duh, and produced in collaboration with Exhibition District. It is located on the exterior of the PWC Building on Balbach St.  Source: Exhibition District, https://www.exhibitiondistrict.com/'

In [121]:
nycphil['properties.description'].iloc[0]

"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>, and produced in collaboration with <a href='https://www.exhibitiondistrict.com/' target='_blank'>Exhibition District</a>. It is located on the exterior of the PWC Building on Balbach St. <br/> Source: Exhibition District, https://www.exhibitiondistrict.com/"

In [122]:
type(soup.find('a'))

bs4.element.Tag

In [132]:
tags = soup.find_all('a')

In [134]:
print(tags)
for t in tags:
    print(t.text, t.attrs['href'])

[<a href="https://www.instagram.com/j.duh_/" target="_blank">J.Duh</a>, <a href="https://www.exhibitiondistrict.com/" target="_blank">Exhibition District</a>]
J.Duh https://www.instagram.com/j.duh_/
Exhibition District https://www.exhibitiondistrict.com/


In [131]:
soup2.absolute_links

In [ ]:
#edit 'properties.artist' column keeping text tag as name and putting attrs into 'properties.artist_site'

In [125]:
with open(fname) as fp:
    soup2 = BeautifulSoup(fp, 'lxml')



In [128]:
len(soup2.find_all('a'))

284

In [ ]:
soup2.absolute_links

# Reading each nest of the json into DF one at a time and saving back to json

In [323]:
#read consolidated-artworks.json into dataframe
#Remember: you had to replace typo herf with href
df = pd.read_json(fname)

In [324]:
#notice type is copied for each record but only one line in original json
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 2 columns):
features    270 non-null object
type        270 non-null object
dtypes: object(2)
memory usage: 6.3+ KB


In [325]:
#features_df is the DF that holds 'features'
features_df=pd.read_json((df['features']).to_json(orient='records'))
features_df.head()

,addToMap,errors,geometry,id,properties,type
0,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88887, 37.32718]}",10667523,"{'title': 'San José', 'artist': '<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.D...",Feature
1,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.97798, 37.30554]}",23925665,"{'title': 'Nothing Is Too Good for the Working Class', 'artist': 'Carlos Rodriguez', 'descriptio...",Feature
2,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88582, 37.33022]}",31154388,"{'title': 'Guardian Tiger', 'artist': '<a href='http://mikeborja.tumblr.com/' target='_blank'>Mi...",Feature
3,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.89577, 37.35107]}",38393353,"{'title': 'Kokeshis de San Jo', 'artist': '<a href='http://samrodriguezart.com/' target='_blank'...",Feature
4,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.84637, 37.34036]}",45973257,"{'title': 'A Monument to the Last Barn', 'artist': 'Gloria Bornstein', 'description': 'A native ...",Feature


In [326]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 6 columns):
addToMap      1 non-null float64
errors        4 non-null object
geometry      270 non-null object
id            270 non-null int64
properties    270 non-null object
type          270 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 14.8+ KB


In [327]:
#covert features_df['properties'] into a json and open into its own df==>properties_df
properties_df=pd.read_json((features_df['properties']).to_json(orient='records'))
properties_df.head()

,address,artist,city,country,crossStreet,description,image,postalCode,sourceOfInformation,sourceURL,state,title,website
0,"488 Almaden Blvd, Ste 1800",<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN
1,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN
2,396 S 1st St.,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/
3,200 E Taylor St,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN
4,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN


In [328]:
#change artist name column to simple txt string getting rid of any html tags
properties_df['artist'] = artist_name
properties_df.head()

,address,artist,city,country,crossStreet,description,image,postalCode,sourceOfInformation,sourceURL,state,title,website
0,"488 Almaden Blvd, Ste 1800",J.Duh,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN
1,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN
2,396 S 1st St.,Mike Borja,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/
3,200 E Taylor St,Sam Rodriguez,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN
4,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN


In [329]:
#convert properties_df back to json and load into a dict
properties_json = properties_df.to_json(orient='records')
properties_json_dict = json.loads(properties_json)

#write the properties to json for QA in pretty format
with open('properties.json', 'w') as f:
    pprint.pprint(properties_json_dict, stream=f)
f.close()
pprint.pprint(properties_json_dict)

[{'address': '488 Almaden Blvd, Ste 1800',
  'artist': 'J.Duh',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by <a '
                 "href='https://www.instagram.com/j.duh_/' "
                 "target='_blank'>J.Duh</a>, and produced in collaboration "
                 "with <a href='https://www.exhibitiondistrict.com/' "
                 "target='_blank'>Exhibition District</a>. It is located on "
                 'the exterior of the PWC Building on Balbach St. <br/> '
                 'Source: Exhibition District, '
                 'https://www.exhibitiondistrict.com/',
  'image': None,
  'postalCode': 95113,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'San José',
  'website': None},
 {'address': '1080 Saratoga Ave #15',
  'artist': 'Carlos Rodriguez',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by artis

  'sourceURL': None,
  'state': 'CA',
  'title': 'Anno Domini Collage',
  'website': None},
 {'address': '280 S 1st St',
  'artist': 'Bunnie Reiss',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': "<a href='http://www.bunniereiss.com/' target='_blank'>Bunnie "
                 'Reiss</a> painted this brightly colored folk mural along the '
                 'exterior wall of the the Robert F. Peckham United States '
                 'Courthouse and Federal Building on S 1st Street. The mural '
                 'was produced in collaboration with Empire Seven Studios. '
                 "<br/><br/> <a href='http://www.bunniereiss.com/empire-7/' "
                 "target='_blank'>Learn more</a>. Photo source: <a "
                 "href='https://www.empiresevenstudios.com/' "
                 "target='_blank'>Empire Seven Studios</a>.",
  'image': 'https://static1.squarespace.com/static/5681ee28841abae3b7d8dec9/5681f5521c12101f97a4de89/5a53b8e8

  'artist': 'Ben Henderson',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by <a '
                 "href='https://www.instagram.com/ohmaigawd/' "
                 "target='_blank'>Ben Henderson</a> and <a "
                 "href='https://www.instagram.com/evilrobot42/' "
                 "target='_blank'>Lacey Bryant</a>. It was commissioned by <a "
                 "href='http://www.parkstationhashery.com/' "
                 "target='_blank'>Park Station Hashery.</a> View a photo of "
                 'the <a '
                 "href='https://www.instagram.com/p/BJCQ8WUBjrV/?taken-by=ohmaigawd' "
                 "target='_blank'>mural</a>. <br/> Source: Ben Henderson, "
                 'https://www.instagram.com/ohmaigawd/',
  'image': None,
  'postalCode': 95126,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Park Station Hashery Mural',
  'website': None},
 {'address': '980 S

  'crossStreet': None,
  'description': 'The San José Public Art Program in association with Adobe '
                 'Systems, commissioned artist Ben Rubin in the creation of '
                 "artwork for Adobe's Amaden Tower. \xa0Not unlike the "
                 'Semaphore Telegraphs of the 18th century,each wheel of the '
                 'San José Semaphore can assume four distinct positions and '
                 'together, the four wheels have a vocabulary of 256 possible '
                 'combinations that turn into new positions every 7.2 seconds '
                 'pulsing out a coded message,\xa0the content of which is a '
                 'mystery left as a challenge\xa0for the viewers to '
                 'decipher.\xa0As of October 2012, a new code is being '
                 'displayed; cracking the encryption technique and deciphering '
                 'the message is posed as a challenge for the public. To the '
                 'first person or group to success

  'website': None},
 {'address': '150 W San Carlos St',
  'artist': 'Soo-in Yang',
  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'Creating a social space bringing together Convention Center '
                 'event goers and the general public, the project’s central '
                 'element is a 40’ diameter canopy, supported by three groups '
                 'of columns. The canopy is created with six interlocking '
                 'steel rings intricately held in tension by steel cables, '
                 "covered by translucent polycarbonates 'leaves.'Critical to "
                 'Yang’s concept for the artwork is the gathering of spoken '
                 'ideas. Adjacent to the Tree is the project’s sculptural '
                 'sound “booth” (the “Seed”) where a sensor activated, audible '
                 'prompt, encourages community to leave a short voice '
                 'message.\xa0 Once a day before morning, messages are '
   

...

  'state': 'CA',
  'title': 'Untitled',
  'website': None},
 {'address': '525 N 7th St',
  'artist': 'Girafa',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural is painted by <a '
                 "href='https://www.instagram.com/longneck4life/' "
                 "target='_blank'>Girafa</a>, and produced in collaboration "
                 "with <a href='https://www.empiresevenstudios.com' "
                 "target='_blank'>Empire Seven Studios</a>, POW!WOW! San José, "
                 "and <a href='https://www.instagram.com/ungramr/' "
                 "target='_blank'>Ungramr</a>. It was exhibited at the 2017 "
                 'POW!WOW! San José festival. <br/> Source: POW! WOW! San '
                 'José, 2017 Mural & Event Locations '
                 'https://www.powwowsanjose.com/2017-locations/',
  'image': None,
  'postalCode': 95112,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Fore

  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'The artwork is inspired by historic images from the P.A.L '
                 'collection and based on the photo of a skinny young boy with '
                 'over-sized gloves supported by stick-thing arms raised over '
                 'his head in triumph. Mentorship, healthy competition and '
                 'community - these are for-grounded in the art project.',
  'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2881%2ftriumph2_201406111806234456.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
  'postalCode': 95116,
  'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
  'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Triumph-San-Jose-Police-Athle-431',
  'state': 'CA',
  'title': 'Triumph',
  'website': None},
 {'address': 'S 1st St & S Market St',
  'artist': 'East Los Streetscape

                 'the world on the artwork’s dynamic display.  \xa0 \xa0 \r\n'
                 '\r\n'
                 'Sponsored by: (Adobe Foundation)',
  'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2475%5c2479%2feCloud_photo-ben.blackwell_201401281407112198.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
  'postalCode': 95110,
  'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
  'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-eCLOUD-Noman-Y-Mineta-San-Jos-404',
  'state': 'CA',
  'title': 'eCLOUD',
  'website': None},
 {'address': '1701 Airport Blvd.',
  'artist': 'Banny Banerjee, Matt Gorbet, Susan LK Gorbet and Margaret Orth',
  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'Explore time through the movement of this suspended '
                 'sculpture that breathes, ripples and reacts to passengers '
                 'wh

                 "class='telerik-reTableOddCol-1'>&nbsp;<b>Artist</b></td></tr><tr "
                 "class='telerik-reTableOddRow-1'><td "
                 "class='telerik-reTableEvenCol-1' style=''>Over the "
                 'Guadalupe<br></td><td '
                 "class='telerik-reTableOddCol-1'>Barbara "
                 "Kasten<br></td></tr><tr class='telerik-reTableEvenRow-1'><td "
                 "class='telerik-reTableEvenCol-1'>Obor<br></td><td "
                 "class='telerik-reTableOddCol-1'>Ed Moses<br></td></tr><tr "
                 "class='telerik-reTableOddRow-1'><td "
                 "class='telerik-reTableEvenCol-1'>Wanderers</td><td "
                 "class='telerik-reTableOddCol-1'>Selina "
                 "Trieff<br></td></tr><tr class='telerik-reTableEvenRow-1'><td "
                 "class='telerik-reTableEvenCol-1'>Honolulu Hay "
                 "Dos<br></td><td class='telerik-reTableOddCol-1'>Rupert "
                 "Garcia<br></td></tr><tr class='

                 'Park in San José. Read this <a '
                 "href='http://www.muralmusicarts.org/blog/nvidia-2012/' "
                 "target='_blank'>blog</a> to learn more. <br/>Image: Yan-Yin "
                 'Choy. Source: MMAP, '
                 'http://www.muralmusicarts.org/blog/nvidia-2012/',
  'image': 'img/UntitledMural_LocatedAtVeggielutionFarm_SanJose_photoby_YanYinChoy.jpg',
  'postalCode': 95116,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Veggielution Farm Outdoor Kitchen Mural',
  'website': None},
 {'address': 'Hwy 87 at Santa Clara St. & Hwy 87 at San Fernando St.',
  'artist': 'Dan Corson',
  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'Sensing YOU and Sensing WATER are two artworks '
                 'geographically and conceptually linked but distinctly\r\n'
                 'different. The projects are part of the “Illuminating '
                 'Downtown Project (IDP).” The inten

In [330]:
#now get the properties json dict for each row(or record) 
#and replace the properties column in the features df
#with updated properties

a=[]
for record in range(len(properties_json_dict)):
    a.append(properties_json_dict[record])
    
features_df['properties']=a
features_df.head()

,addToMap,errors,geometry,id,properties,type
0,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88887, 37.32718]}",10667523,"{'address': '488 Almaden Blvd, Ste 1800', 'artist': 'J.Duh', 'city': 'San José', 'country': 'Uni...",Feature
1,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.97798, 37.30554]}",23925665,"{'address': '1080 Saratoga Ave #15', 'artist': 'Carlos Rodriguez', 'city': 'San José', 'country'...",Feature
2,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88582, 37.33022]}",31154388,"{'address': '396 S 1st St.', 'artist': 'Mike Borja', 'city': 'San José', 'country': 'United Stat...",Feature
3,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.89577, 37.35107]}",38393353,"{'address': '200 E Taylor St', 'artist': 'Sam Rodriguez', 'city': 'San José', 'country': 'United...",Feature
4,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.84637, 37.34036]}",45973257,"{'address': '647 S King Rd', 'artist': 'Gloria Bornstein', 'city': 'San José', 'country': 'Unite...",Feature


In [331]:
#convert features df into json
features_json = features_df.to_json(orient='records')

#load features json into a dict
features_json_dict = json.loads(features_json)

#write features json dict to a json file in pretty format for QA and for later use
with open('features.json', 'w') as f:
    pprint.pprint(features_json_dict, stream=f)
f.close()
pprint.pprint(tmp2)

[{'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.88887, 37.32718], 'type': 'Point'},
  'id': 10667523,
  'properties': {'address': '488 Almaden Blvd, Ste 1800',
                 'artist': 'J.Duh',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'This mural was painted by <a '
                                "href='https://www.instagram.com/j.duh_/' "
                                "target='_blank'>J.Duh</a>, and produced in "
                                'collaboration with <a '
                                "href='https://www.exhibitiondistrict.com/' "
                                "target='_blank'>Exhibition District</a>. It "
                                'is located on the exterior of the PWC '
                                'Building on Balbach St. <br/> Source: '
                                'Exhibition District, '
                        

                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.9139669, 37.3522017], 'type': 'Point'},
  'id': 332981421,
  'properties': {'address': '1701 Airport Blvd.',
                 'artist': 'Carlos Perez',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'This illustrated wall installation recognizes '
                                'three prime-movers behind San Jose’s airport: '
                                'Ernie Renzel, James Nissen and Norman Y. '
                                'Mineta.',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2891%2fminetacompfin_201406111912324378.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                 'postalCode': 95110,
                 'sourceOfInformation': 'http://sa

                 'title': 'Urban Rain 1',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.8864981049, 37.3354988828],
               'type': 'Point'},
  'id': 556438453,
  'properties': {'address': '150 E San Fernando St.',
                 'artist': 'Mel Chin',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'This collection of 34 artworks is sited '
                                'throughout the MLK Library. These '
                                'site-specific artworks are designed to '
                                "provoke viewers' interest and curiosity, "
                                'while encouraging\xa0exploration and '
                                'circulation throughout the Library. The '
                                'artworks vary from large and dramatic '
                               

                 'sourceOfInformation': None,
                 'sourceURL': None,
                 'state': 'CA',
                 'title': 'Virgin de Guadalupe',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.8867007, 37.3306511], 'type': 'Point'},
  'id': 666599470,
  'properties': {'address': '349 S 1st St',
                 'artist': 'Royyal Dog',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'Mural of legendary singer and songwriter '
                                'Sade. Sade (Helen Folasade Adu or Sade Adu) '
                                'is the lead vocalist of the iconic British '
                                'band Sade, and a British Nigerian singer and '
                                'songwriter. Royyal Dog painted this mural at '
                                'the back of The 

                                'plants and historic artifacts painted\xa0in '
                                'silver-leaf. The grapevine is a reminder of '
                                'an earlier use of the\xa0neighboring land and '
                                'the specific flowers were chosen on the\xa0'
                                'basis of their geographic origin, a reference '
                                'to the international\xa0nature of San '
                                'José.\r\n'
                                'The Wall of Wonder is a cut steel\xa0mural in '
                                "the Children's Area, suggesting the\xa0"
                                "limitless potential of a child's imagination "
                                'and the\xa0role the library can play in '
                                'encouraging exploration.',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-000

...

                 'sourceURL': None,
                 'state': 'CA',
                 'title': 'Gemini Twins',
                 'website': 'http://mikeborja.tumblr.com/'},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.888989981, 37.3318121176], 'type': 'Point'},
  'id': 3279237363,
  'properties': {'address': '201 S. Market Street',
                 'artist': 'David Bottini',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'This bright geometric abstract piece is '
                                'located by the Tech Museum on Park Avenue. It '
                                'was created by contemporary Northern '
                                'California sculptor David Bottini who works '
                                'in the styles of pure abstraction and '
                                'neo-constructivism.',
                 'image': 'h

  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.8918049668, 37.3300339828],
               'type': 'Point'},
  'id': 3456689603,
  'properties': {'address': '255 S Almaden Blvd',
                 'artist': 'Various',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': "<table class='telerik-reTable-1' "
                                "style='width: 100%;' cellpadding='2' "
                                "cellspacing='1'><tbody><tr "
                                "class='telerik-reTableEvenRow-1'><td "
                                "class='telerik-reTableEvenCol-1'><b>Title</b><br></td><td "
                                "class='telerik-reTableOddCol-1'>&nbsp;<b>Artist</b></td></tr><tr "
                                "class='telerik-reTableOddRow-1'><td "
                                "class='telerik-reTableEvenCol-1' "
                    

                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.87376, 37.32312], 'type': 'Point'},
  'id': 3913083978,
  'properties': {'address': '1035 S 6th St',
                 'artist': 'Francisco Ramirez',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'This mural of butterflies, gears and hands is '
                                'located at Art Ark Gallery. <br/> Source: '
                                'Franciso Ramirez, '
                                'https://www.instagram.com/fco1980/.',
                 'image': None,
                 'postalCode': 95112,
                 'sourceOfInformation': None,
                 'sourceURL': None,
                 'state': 'CA',
                 'title': 'Untitled Mural',
                 'website': 'https://www.instagram.com/fco1980/'},
  'type': 'Feature'},
 

                                'Source: Paul J. Gonzalez Art Studios, '
                                'http://www.pauljgonzalezartstudios.com/',
                 'image': None,
                 'postalCode': 94087,
                 'sourceOfInformation': None,
                 'sourceURL': None,
                 'state': 'CA',
                 'title': 'Mural at Peterson Middle School',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.9154323, 37.3343304], 'type': 'Point'},
  'id': 4097625919,
  'properties': {'address': '1460 The Alameda',
                 'artist': 'Bill Gould and Glen Rogers',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'Spirit Gate creates\xa0an entry transition '
                                'between The Alameda and the school building. '
                                'The artists

                                'Forming; bent piece by piece and fit into a '
                                'notched support structure after which it was '
                                'galvanized for its final finish. <br/><br/> '
                                'The artwork is titled “Lupe,” but has quickly '
                                'assumed the nickname Lupe the Mammoth as a '
                                'popular post on social media; '
                                '#LupeTheMammoth. <br/><br/> Source: City of '
                                'San José Public Art Program, '
                                'http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-Lupe-the-Mammoth-Guadalupe-Tr-559',
                 'image': None,
                 'postalCode': 95113,
                 'sourceOfInformation': None,
                 'sourceURL': None,
                 'state': 'CA',
                 'title': 'Lupe the Mammoth',
                 'website': None

# don't do this b/c the resultant df doesn't give back original json format

#now get the features json dict for each row(or record) 
#and replace the features column in df
#with updated features

a=[]
for record in range(len(features_json_dict)):
    a.append(features_json_dict[record])
    
df['features']=a
df.head()

# Load original json into a dict and replace features data with features_json_dict

In [332]:
with open(fname) as f:
    original_dict = json.load(f)
f.close()

original_dict['features'] = features_json_dict

with open('consolidated-artworks-fixed.json', 'w') as f:
    pprint.pprint(original_dict, stream=f)
f.close()

#used original_dict.keys()